In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#!git clone https://github.com/Hikarukurosawa123/TUPIL_Kidney.git
#!git clone https://github.com/Hikarukurosawa123/TUPIL_Kidney.git drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney
#%cd drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney
#!git stash

#!git add .
#!git commit -m "Added latest Colab notebook"
#!git push origin main

#%cd /content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney
#!git stash

!git pull origin main     # or "master", depending on the branch name


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, regularizers
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from skimage.transform import resize
from collections import defaultdict

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims
import scipy.ndimage
from sklearn.utils import class_weight
# Flexible input configuration - specify which input types to use
# Options: 'B_mode', 'MBF', 'SI' - can use any combination
INPUT_TYPES = ['B_mode', 'MBF', 'SI']  # Change this to specify which inputs to use
# Examples:
# INPUT_TYPES = ['B_mode']  # Only B-mode
# INPUT_TYPES = ['B_mode', 'MBF']  # B-mode + MBF
# INPUT_TYPES = ['B_mode', 'SI']  # B-mode + SI
# INPUT_TYPES = ['MBF', 'SI']  # MBF + SI
# INPUT_TYPES = ['B_mode', 'MBF', 'SI']  # All three


# Environment detection
def is_google_colab():
    """Check if running in Google Colab environment"""
    try:
        import google.colab
        return True
    except ImportError:
        return False


def is_google_drive_mounted():
    """Check if Google Drive is mounted in Colab"""
    return os.path.exists('/content/drive/MyDrive')
# Set paths based on environment
if is_google_colab() and is_google_drive_mounted():
    # Google Colab with Google Drive mounted
    IMAGE_FOLDERS = {
        'B_mode': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Bmode',
        'MBF': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/MBF',
        'SI': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/SI'
    }
    CSV_FILE = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/csv/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running on Google Colab with Google Drive mounted")
elif is_google_colab():
    # Google Colab without Google Drive mounted
    IMAGE_FOLDERS = {
        'B_mode': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Bmode',
        'MBF': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/MBF',
        'SI': '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/SI'
    }
    CSV_FILE = '/content/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/content/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running on Google Colab without Google Drive mounted")
else:
    # Local environment
    IMAGE_FOLDERS = {
        'B_mode': '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/Bmode_resize',
        'MBF': '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/MBF',
        'SI': '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/SI'
    }
    CSV_FILE = '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/csv/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/data/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running locally")

# Print selected input types and their paths
print(f"Selected input types: {INPUT_TYPES}")
for input_type in INPUT_TYPES:
    print(f"{input_type}_FOLDER: {IMAGE_FOLDERS[input_type]}")
print(f"CSV_FILE: {CSV_FILE}")
print(f"MODEL_WEIGHTS_PATH: {MODEL_WEIGHTS_PATH}")

BATCH_SIZE = 16 # batch = 8 -> 10% batch = 16 --> doesnt work
EPOCHS = 160 # epoch = 40 --> 10% epoch = 50 --> 9%
SEED = 42


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, regularizers
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from skimage.transform import resize
from collections import defaultdict

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims
import scipy.ndimage
from sklearn.utils import class_weight

# Environment detection
def is_google_colab():
    """Check if running in Google Colab environment"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

def is_google_drive_mounted():
    """Check if Google Drive is mounted in Colab"""
    return os.path.exists('/content/drive/MyDrive')

# Set paths based on environment
if is_google_colab() and is_google_drive_mounted():
    # Google Colab with Google Drive mounted
    B_MODE_IMAGE_FOLDER = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Bmode'
    H_SCAN_IMAGE_FOLDER = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Hscan' # currently resized image input (224 x 224)

    CSV_FILE = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/csv/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running on Google Colab with Google Drive mounted")
elif is_google_colab():
    # Google Colab without Google Drive mounted
    B_MODE_IMAGE_FOLDER = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Bmode'
    H_SCAN_IMAGE_FOLDER = '/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney/data/Hscan' # currently resized image input (224 x 224)

    CSV_FILE = '/content/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/content/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running on Google Colab without Google Drive mounted")
else:
    # Local environment
    B_MODE_IMAGE_FOLDER = '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/Bmode'
    H_SCAN_IMAGE_FOLDER = 'Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/Hscan' # currently resized image input (224 x 224)

    CSV_FILE = '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/csv/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv'
    MODEL_WEIGHTS_PATH = '/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/data/model_weights/RadImageNet-ResNet50_notop.h5'
    print("Running locally")

print(f"IMAGE_FOLDER: {B_MODE_IMAGE_FOLDER}")
print(f"CSV_FILE: {CSV_FILE}")
print(f"MODEL_WEIGHTS_PATH: {MODEL_WEIGHTS_PATH}")

BATCH_SIZE = 16 # batch = 8 -> 10% batch = 16 --> doesnt work
EPOCHS = 160 # epoch = 40 --> 10% epoch = 50 --> 9%
SEED = 42

Running locally
IMAGE_FOLDER: /Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/data/lanczos_shape_corrected_only_nc_resized_images
CSV_FILE: /Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/csv/patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv
MODEL_WEIGHTS_PATH: /Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney/data/model_weights/RadImageNet-ResNet50_notop.h5


In [ ]:
# Test environment detection and path setup
print("=== Environment Detection Test ===") 
print(f"is_google_colab(): {is_google_colab()}")
print(f"is_google_drive_mounted(): {is_google_drive_mounted()}")
print(f"Current working directory: {os.getcwd()}")
print(f"IMAGE_FOLDER exists: {os.path.exists(B_MODE_IMAGE_FOLDER)}")
print(f"IMAGE_FOLDER exists: {os.path.exists(H_SCAN_IMAGE_FOLDER)}")
print(f"CSV_FILE exists: {os.path.exists(CSV_FILE)}")
print(f"MODEL_WEIGHTS_PATH exists: {os.path.exists(MODEL_WEIGHTS_PATH)}")
print("=== End Test ===")


=== Environment Detection Test ===
is_google_colab(): False
is_google_drive_mounted(): False
Current working directory: /Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney
IMAGE_FOLDER exists: True
CSV_FILE exists: True
MODEL_WEIGHTS_PATH exists: True
=== End Test ===


In [ ]:
# Flexible Patient class for variable number of input types
class Patient:
    def __init__(self, patient_id, egfr, egfr_val, image_paths_dict):
        self.patient_id = patient_id
        self.egfr = egfr #label
        self.egfr_val = egfr_val #actual value
        self.image_paths_dict = image_paths_dict  # Dictionary with input_type -> list of paths
    
    def get_paths(self, input_type):
        """Get paths for a specific input type"""
        return self.image_paths_dict.get(input_type, [])
    
    def has_all_inputs(self, required_input_types):
        """Check if patient has all required input types"""
        return all(len(self.get_paths(input_type)) > 0 for input_type in required_input_types)


def label_img_classification_by_patient_flexible(input_types, image_folders, csv_file):
    """
    Returns a list of Patient objects with flexible input types.
    
    Args:
        input_types: List of input types to use (e.g., ['B_mode', 'MBF', 'SI'])
        image_folders: Dictionary mapping input_type -> folder_path
        csv_file: Path to CSV file with patient data
    """
    eGFR_data = pd.read_csv(csv_file)
    eGFR_data.rename(columns={'Patient ID': 'patient_id', 'eGFR (abs/closest)': 'eGFR'}, inplace=True)
    eGFR_data['patient_id'] = eGFR_data['patient_id'].astype(int)
    eGFR_data.set_index('patient_id', inplace=True)

    # Initialize patient image map with all input types
    patient_image_map = defaultdict(lambda: {input_type: [] for input_type in input_types})

    # Helper function to extract patient ID from filename
    def extract_patient_id(filename, input_type):
        try:
            if input_type == "B_mode":
                # Format: Patient_100_Resized_Image_1.png
                return int(filename.split('_')[1])
            else:
                # Format: P100_PTONR_01_Image_1_rf_MBF_resized.png or P100_PTONR_01_Image_1_rf_SI_resized.png
                return int(filename.split('_')[0][1:])  # Remove 'P' prefix
        except:
            return None

    # Process files for each input type
    for input_type in input_types:
        if input_type in image_folders:
            folder_path = image_folders[input_type]
            if os.path.exists(folder_path):
                files = sorted(os.listdir(folder_path))
                for filename in files:
                    patient_id = extract_patient_id(filename, input_type)
                    if patient_id and patient_id in eGFR_data.index:
                        patient_image_map[patient_id][input_type].append(os.path.join(folder_path, filename))
            else:
                print(f"Warning: Folder {folder_path} does not exist for input type {input_type}")

    # Build Patient objects - only include patients with all required input types
    patient_objects = []
    for patient_id, paths_dict in patient_image_map.items():
        # Check if patient has all required input types
        has_all_types = all(len(paths_dict[input_type]) > 0 for input_type in input_types)
        
        if has_all_types:
            egfr = eGFR_data.loc[patient_id, 'eGFR']
            egfrLabel = 1 if egfr >= 60 else 0
            patient_objects.append(Patient(
                patient_id,
                egfrLabel,
                egfr,
                paths_dict
            ))

    return patient_objects


# Prepare image preprocessing function
def preprocess_img(img_path):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)  # RGB image
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Convert TensorFlow tensor to NumPy array for compatibility with ImageDataGenerator
    return tf.keras.utils.img_to_array(image)


In [ ]:
import tensorflow as tf

def create_dataset_from_patients_flexible(patients, input_types, augment=False, batch_size=4):
    """
    Creates a tf.data.Dataset for flexible input images.
    
    Args:
        patients: List of Patient objects
        input_types: List of input types to use (e.g., ['B_mode', 'MBF', 'SI'])
        augment: Whether to apply data augmentation
        batch_size: Batch size for the dataset
    
    Returns:
        tf.data.Dataset with flexible inputs
    """
    
    # Load images for all input types
    image_lists = {input_type: [] for input_type in input_types}
    labels = []
    
    for patient in patients:
        # Get the minimum number of images across all input types for this patient
        min_images = min(len(patient.get_paths(input_type)) for input_type in input_types)
        
        for i in range(min_images):
            # Load images for each input type
            for input_type in input_types:
                image_lists[input_type].append(preprocess_img(patient.get_paths(input_type)[i]))
            labels.append(patient.egfr)
    
    # Create dataset based on number of input types
    if len(input_types) == 1:
        # Single input
        input_type = input_types[0]
        dataset = tf.data.Dataset.from_tensor_slices((image_lists[input_type], labels))
    else:
        # Multiple inputs - create tuple of inputs
        input_tuples = list(zip(*[image_lists[input_type] for input_type in input_types]))
        dataset = tf.data.Dataset.from_tensor_slices((input_tuples, labels))
    
    dataset = dataset.batch(batch_size)
    
    if augment:
        # Define augmentation pipeline
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.25),
            tf.keras.layers.RandomZoom(0.1),
        ])
        
        if len(input_types) == 1:
            # Single input augmentation
            def augment_single(inputs, label):
                augmented = data_augmentation(inputs, training=True)
                return augmented, label
            dataset = dataset.map(augment_single, num_parallel_calls=tf.data.AUTOTUNE)
        else:
            # Multiple input augmentation - apply same transformation to all inputs
            def augment_multiple(inputs, label):
                # Concatenate all inputs along channel axis for synchronized augmentation
                combined = tf.concat(inputs, axis=-1)  # H x W x (3 * num_inputs)
                combined = data_augmentation(combined, training=True)
                
                # Split back into separate inputs
                split_inputs = []
                for i in range(len(input_types)):
                    start_channel = i * 3
                    end_channel = (i + 1) * 3
                    split_inputs.append(combined[..., start_channel:end_channel])
                
                return tuple(split_inputs), label
            
            dataset = dataset.map(augment_multiple, num_parallel_calls=tf.data.AUTOTUNE)
    
    return dataset.prefetch(tf.data.AUTOTUNE)


def summarize_patients_flexible(patients, input_types):
    """Summarize the flexible input patient data"""
    num_patients = len(patients)
    
    print(f"Number of patients: {num_patients}")
    print(f"Input types: {input_types}")
    
    # Count images for each input type
    for input_type in input_types:
        total_images = sum(len(patient.get_paths(input_type)) for patient in patients)
        print(f"Total {input_type} images: {total_images}")
        
        # Distribution of images per patient
        counts = [len(patient.get_paths(input_type)) for patient in patients]
        print(f"{input_type} - Min: {min(counts)}, Max: {max(counts)}, Avg: {np.mean(counts):.2f}")
    
    return num_patients


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_resnet_model_flexible(input_types, with_transfer_learning=True):
    """
    Build ResNet model for flexible input types.
    
    Args:
        input_types: List of input types (e.g., ['B_mode', 'MBF', 'SI'])
        with_transfer_learning: Whether to use transfer learning
    
    Returns:
        Compiled Keras model
    """
    
    if len(input_types) == 1:
        # Single input model
        input_type = input_types[0]
        input_layer = layers.Input(shape=(224, 224, 3), name=f'{input_type}_input')
        inputs = input_layer
        
        # Load base model
        if with_transfer_learning:
            base_model = models.load_model(MODEL_WEIGHTS_PATH, compile=False)
        else:
            base_model = tf.keras.applications.ResNet50(
                weights=None, 
                include_top=False, 
                input_shape=(224, 224, 3)
            )
        
        base_model.trainable = True
        x = base_model(inputs)
        
    else:
        # Multiple input model
        input_layers = []
        for input_type in input_types:
            input_layer = layers.Input(shape=(224, 224, 3), name=f'{input_type}_input')
            input_layers.append(input_layer)
        
        # Load base model
        if with_transfer_learning:
            base_model = models.load_model(MODEL_WEIGHTS_PATH, compile=False)
        else:
            base_model = tf.keras.applications.ResNet50(
                weights=None, 
                include_top=False, 
                input_shape=(224, 224, 3)
            )
        
        base_model.trainable = True
        
        # Process each input through the base model
        feature_maps = []
        for input_layer in input_layers:
            features = base_model(input_layer)
            pooled = layers.GlobalAveragePooling2D()(features)
            feature_maps.append(pooled)
        
        # Concatenate features from all inputs
        x = layers.Concatenate()(feature_maps)
    
    # Global average pooling for single input
    if len(input_types) == 1:
        x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers for classification
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    # Final binary classification
    output = layers.Dense(1, activation='sigmoid')(x)
    
    # Create model
    if len(input_types) == 1:
        model = models.Model(inputs=inputs, outputs=output)
    else:
        model = models.Model(inputs=input_layers, outputs=output)
    
    return model


def build_resnet_model_flexible_concatenated(input_types, with_transfer_learning=True):
    """
    Alternative architecture: concatenate inputs first, then process through single model.
    
    Args:
        input_types: List of input types (e.g., ['B_mode', 'MBF', 'SI'])
        with_transfer_learning: Whether to use transfer learning
    
    Returns:
        Compiled Keras model
    """
    
    if len(input_types) == 1:
        # Single input - same as regular model
        return build_resnet_model_flexible(input_types, with_transfer_learning)
    
    # Multiple inputs - concatenate along channel axis
    input_layers = []
    for input_type in input_types:
        input_layer = layers.Input(shape=(224, 224, 3), name=f'{input_type}_input')
        input_layers.append(input_layer)
    
    # Concatenate inputs along channel axis (224, 224, 3*num_inputs)
    concatenated_input = layers.Concatenate(axis=-1)(input_layers)
    
    # Load base model
    if with_transfer_learning:
        base_model = models.load_model(MODEL_WEIGHTS_PATH, compile=False)
    else:
        base_model = tf.keras.applications.ResNet50(
            weights=None, 
            include_top=False, 
            input_shape=(224, 224, 3 * len(input_types))
        )
    
    base_model.trainable = True
    
    # Process concatenated input
    x = base_model(concatenated_input)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    # Final binary classification
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=input_layers, outputs=output)
    return model




In [ ]:
# --- RAN56 Model ---
import tensorflow as tf
from tensorflow.keras import layers, models
# ----------------------
# Residual Unit
# ----------------------
def residual_unit(x, filters, name=None):
    shortcut = x
    x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Match shortcut channels if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

# ----------------------
# Attention Module
# ----------------------
def attention_module(inputs, filters, name=None):
    # ===== Trunk Branch =====
    trunk = residual_unit(inputs, filters)
    trunk = residual_unit(trunk, filters)

    # ===== Mask Branch =====
    # Downsampling path
    mp1 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(inputs)
    mp1 = residual_unit(mp1, filters)

    mp2 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(mp1)
    mp2 = residual_unit(mp2, filters)

    # Upsampling path
    up1 = layers.UpSampling2D(size=(2, 2))(mp2)
    up1 = residual_unit(up1, filters)
    up1 = layers.Add()([up1, mp1])  # skip connection

    up2 = layers.UpSampling2D(size=(2, 2))(up1)
    up2 = residual_unit(up2, filters)

    # Mask output
    mask = layers.Conv2D(filters, (1, 1), padding='same', activation='sigmoid')(up2)

    # ===== Combine Mask + Trunk =====
    out = layers.Multiply()([mask, trunk])
    out = layers.Add()([out, trunk])  # (1 + mask) * trunk
    out = residual_unit(out, filters)

    return out
def RAN56(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)

    # Stage 1
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)

    # Stage 2
    x = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    att1 = attention_module(x, 256)
    skip1 = att1

    # Stage 3
    x = layers.Conv2D(128, (1, 1), strides=2, padding='same', activation='relu')(att1)
    x = layers.Conv2D(512, (1, 1), padding='same', activation='relu')(x)
    att2 = attention_module(x, 512)
    skip2 = att2

    # Stage 4
    x = layers.Conv2D(256, (1, 1), strides=2, padding='same', activation='relu')(att2)
    x = layers.Conv2D(1024, (3, 3), padding='same', activation='relu')(x)
    att3 = attention_module(x, 1024)
    skip3 = att3

    # Stage 5
    x = layers.Conv2D(512, (1, 1), strides=2, padding='same', activation='relu')(att3)
    x = layers.Conv2D(2048, (3, 3), padding='same', activation='relu')(x)

    # Combine skip connections
    x = layers.Add()([x, skip1, skip2, skip3])

    # Global average pooling and classifier
    x = layers.GlobalAveragePooling2D()(x)
     # Flatten feature and classifier head
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(100, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    #outputs = layers.Dense(num_classes, activation='softmax')(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, output, name='RAN56')
    return model

# Build the model
model = RAN56(input_shape=(224, 224, 3))
model.summary()

In [1]:
def plotAndReturnValidationTesting_flexible(val_patients, model, input_types):
    """Updated validation testing function for flexible input models"""
    patient_to_true_labels = {}
    patient_to_predicted_probs_list = {}

    all_true_labels_individual = []
    all_predicted_probs_individual = []

    # ================= Collect predictions =================
    for patient in val_patients:
        patient_dataset = create_dataset_from_patients_flexible([patient], input_types, augment=False, batch_size=BATCH_SIZE)
        for inputs, labels in patient_dataset:
            true_label = labels[0].numpy()  # Single label per patient
            predictions = model.predict(inputs).flatten()

            patient_to_true_labels[patient.patient_id] = true_label
            patient_to_predicted_probs_list[patient.patient_id] = predictions.tolist()

            # Collect image-level predictions for metrics
            all_true_labels_individual.extend(labels.numpy().flatten())
            all_predicted_probs_individual.extend(predictions)

    # ================= Image-level ROC =================
    all_predicted_labels_individual = [1 if prob >= 0.5 else 0 for prob in all_predicted_probs_individual]
    fpr_img, tpr_img, _ = roc_curve(all_true_labels_individual, all_predicted_probs_individual)
    auc_img = roc_auc_score(all_true_labels_individual, all_predicted_probs_individual)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_img, tpr_img, label=f"Image-level ROC (AUC = {auc_img:.4f})", color='blue')
    plt.plot([0, 1], [0, 1], 'r--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"Image-level ROC Curve - {', '.join(input_types)} Input")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    # ================= Patient-level ROC =================
    patient_true = []
    patient_probs = []

    for pid in patient_to_predicted_probs_list:
        patient_true.append(patient_to_true_labels[pid])
        patient_probs.append(np.mean(patient_to_predicted_probs_list[pid]))  # average across images

    fpr_pat, tpr_pat, _ = roc_curve(patient_true, patient_probs)
    auc_pat = roc_auc_score(patient_true, patient_probs)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_pat, tpr_pat, label=f"Patient-level ROC (AUC = {auc_pat:.4f})", color='green')
    plt.plot([0, 1], [0, 1], 'r--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"Patient-level ROC Curve - {', '.join(input_types)} Input")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    return patient_to_true_labels, patient_to_predicted_probs_list


# Updated hyperparameter tuning configurations for flexible input
def resnet_flexible_branched(input_types):
    """Flexible input with separate branches"""
    model = build_resnet_model_flexible(input_types, with_transfer_learning=True)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

def resnet_flexible_concatenated(input_types):
    """Flexible input with concatenated channels"""
    model = build_resnet_model_flexible_concatenated(input_types, with_transfer_learning=True)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

def resnet_flexible_no_transfer(input_types):
    """Flexible input without transfer learning"""
    model = build_resnet_model_flexible(input_types, with_transfer_learning=False)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

# Create hyperparameter configurations based on selected input types
def create_hyperparameter_configs(input_types):
    """Create hyperparameter configurations for the selected input types"""
    return {
        f'resnet_flexible_branched_{"_".join(input_types)}': lambda: resnet_flexible_branched(input_types),
        f'resnet_flexible_concatenated_{"_".join(input_types)}': lambda: resnet_flexible_concatenated(input_types),
        f'resnet_flexible_no_transfer_{"_".join(input_types)}': lambda: resnet_flexible_no_transfer(input_types)
    }


In [2]:
def plotHistogramOfDataset(training_labels, validation_labels, n_bins):
  plt.hist(training_labels, bins=n_bins, alpha=0.5, label='Training Data', color='blue')
  plt.hist(validation_labels, bins=n_bins, alpha=0.5, label='Validation Data', color='red')
  plt.xlabel('eGFR')
  plt.ylabel('Frequency')
  plt.title(f'Histogram with {n_bins} bins')
  plt.legend()
  plt.show()

def plotTrainingHistory(history):
  # Plot training and validation loss
  plt.figure(figsize=(10, 6))
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Binary Crossentropy Loss')
  plt.title('Training and Validation Loss')
  plt.legend()
  plt.show()

  # Plot Accuracy
  plt.figure(figsize=(10, 6))
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.title('Training and Validation Accuracy')
  plt.legend()
  plt.show()
def plotAndReturnValidationTesting(val_patients, model):
    patient_to_true_labels = {}
    patient_to_predicted_probs_list = {}

    all_true_labels_individual = []
    all_predicted_probs_individual = []

    # ================= Collect predictions =================
    for patient in val_patients:
        patient_dataset = create_dataset_from_patients_flexible([patient], augment=False, batch_size=BATCH_SIZE)
        for images, labels in patient_dataset:
            true_label = labels[0].numpy()  # Single label per patient
            predictions = model.predict(images).flatten()

            patient_to_true_labels[patient.patient_id] = true_label
            patient_to_predicted_probs_list[patient.patient_id] = predictions.tolist()

            # Collect image-level predictions for metrics
            all_true_labels_individual.extend(labels.numpy().flatten())
            all_predicted_probs_individual.extend(predictions)

    # ================= Image-level ROC =================
    all_predicted_labels_individual = [1 if prob >= 0.5 else 0 for prob in all_predicted_probs_individual]
    fpr_img, tpr_img, _ = roc_curve(all_true_labels_individual, all_predicted_probs_individual)
    auc_img = roc_auc_score(all_true_labels_individual, all_predicted_probs_individual)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_img, tpr_img, label=f"Image-level ROC (AUC = {auc_img:.4f})", color='blue')
    plt.plot([0, 1], [0, 1], 'r--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Image-level ROC Curve")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    # ================= Patient-level ROC =================
    patient_true = []
    patient_probs = []

    for pid in patient_to_predicted_probs_list:
        patient_true.append(patient_to_true_labels[pid])
        patient_probs.append(np.mean(patient_to_predicted_probs_list[pid]))  # average across images

    fpr_pat, tpr_pat, _ = roc_curve(patient_true, patient_probs)
    auc_pat = roc_auc_score(patient_true, patient_probs)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_pat, tpr_pat, label=f"Patient-level ROC (AUC = {auc_pat:.4f})", color='green')
    plt.plot([0, 1], [0, 1], 'r--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Patient-level ROC Curve")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    return patient_to_true_labels, patient_to_predicted_probs_list


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import tensorflow as tf

N_RUNS = 5
val_aucs, test_aucs = [], []

for run_idx in range(N_RUNS):
    print(f"\n{'='*40}")
    print(f"▶️ HOLD-OUT RUN {run_idx+1}/{N_RUNS}")
    print(f"{'='*40}\n")

    # Load patients (if not already loaded)
    print("Loading patients with flexible input...")
    all_patients = label_img_classification_by_patient_flexible(
        INPUT_TYPES, 
        IMAGE_FOLDERS, 
        CSV_FILE
    )
    print(f"Total patients with all selected input types: {len(all_patients)}")

    # Summarize once (optional to skip inside loop if identical)
    summarize_patients_flexible(all_patients, INPUT_TYPES)

    # Split into train/val/test with new random state each run
    SEED_RUN = SEED + run_idx  # different seed each run
    train_and_val_patients, test_patients = train_test_split(
        all_patients, test_size=0.1, random_state=SEED_RUN
    )
    train_patients, val_patients = train_test_split(
        train_and_val_patients, test_size=0.2, random_state=SEED_RUN
    )

    print(f"Training patients: {len(train_patients)}")
    print(f"Validation patients: {len(val_patients)}")
    print(f"Test patients: {len(test_patients)}")

    # Create datasets
    train_dataset = create_dataset_from_patients_flexible(
        train_patients, INPUT_TYPES, augment=True, batch_size=BATCH_SIZE
    )
    val_dataset = create_dataset_from_patients_flexible(
        val_patients, INPUT_TYPES, augment=False, batch_size=BATCH_SIZE
    )
    test_dataset = create_dataset_from_patients_flexible(
        test_patients, INPUT_TYPES, augment=False, batch_size=BATCH_SIZE
    )

    # Compute class weights
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(np.array([p.egfr for p in train_patients]).astype(int)),
        y=np.array([p.egfr for p in train_patients]).astype(int)
    )
    class_weights_dict = dict(enumerate(weights))
    print(f"Class weights: {class_weights_dict}")

    # Get model configuration
    hyperparameter_configs = create_hyperparameter_configs(INPUT_TYPES)
    MODEL_CONFIG_NAME = f'resnet_flexible_branched_{"_".join(INPUT_TYPES)}'
    print(f"Using model configuration: {MODEL_CONFIG_NAME}")

    # Build and compile model
    model = hyperparameter_configs[MODEL_CONFIG_NAME]()

    # Early stopping and LR scheduler
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', mode='max', patience=20, restore_best_weights=True
    )
    def step_decay(epoch, lr):
        drop_rate = 0.5
        drop_every = 15
        if epoch > 0 and epoch % drop_every == 0:
            return lr * drop_rate
        return lr
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=0)

    print("Starting training...")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[early_stopping, lr_scheduler],
        class_weight=class_weights_dict,
        verbose=1
    )

    # === Evaluate Validation ===
    print("\nEvaluating on validation set...")
    val_true, val_pred = [], []
    patient_to_true_labels, patient_to_predicted_probs = plotAndReturnValidationTesting_flexible(val_patients, model, INPUT_TYPES)
    for patient_id in patient_to_predicted_probs:
        for prob in patient_to_predicted_probs[patient_id]:
            val_true.append(patient_to_true_labels[patient_id])
            val_pred.append(prob)
    val_auc = roc_auc_score(val_true, val_pred)
    val_aucs.append(val_auc)
    print(f"Validation AUC (Run {run_idx+1}): {val_auc:.4f}")

    # === Evaluate Test ===
    print("\nEvaluating on test set...")
    test_true, test_pred = [], []
    patient_to_true_labels, patient_to_predicted_probs = plotAndReturnValidationTesting_flexible(test_patients, model, INPUT_TYPES)
    for patient_id in patient_to_predicted_probs:
        for prob in patient_to_predicted_probs[patient_id]:
            test_true.append(patient_to_true_labels[patient_id])
            test_pred.append(prob)
    test_auc = roc_auc_score(test_true, test_pred)
    test_aucs.append(test_auc)
    print(f"Test AUC (Run {run_idx+1}): {test_auc:.4f}")

    print(classification_report(test_true, [1 if p >= 0.5 else 0 for p in test_pred], digits=4))


# ===========================
# ✅ Final Summary of 5 Runs
# ===========================
print("\n" + "="*60)
print("FINAL SUMMARY OVER MULTIPLE HOLD-OUT RUNS")
print("="*60)
print(f"Validation AUCs: {['%.4f' % a for a in val_aucs]}")
print(f"Test AUCs:       {['%.4f' % a for a in test_aucs]}")
print(f"\nAverage Validation AUC: {np.mean(val_aucs):.4f} ± {np.std(val_aucs):.4f}")
print(f"Average Test AUC:       {np.mean(test_aucs):.4f} ± {np.std(test_aucs):.4f}")


Loading patients with flexible input...


NameError: name 'label_img_classification_by_patient_flexible' is not defined

In [ ]:
# Example: How to easily switch between different model configurations
# This cell demonstrates how to use the hyperparameter configurations

print("=== Model Configuration Examples ===")
print()

# Get all available configurations for current INPUT_TYPES
hyperparameter_configs = create_hyperparameter_configs(INPUT_TYPES)
print(f"Available configurations for {INPUT_TYPES}:")
for config_name in hyperparameter_configs.keys():
    print(f"  - {config_name}")

print()
print("To use a different model architecture, simply change the MODEL_CONFIG_NAME variable:")
print()

# Example 1: Branched architecture (default)
print("# Example 1: Branched architecture")
print(f"MODEL_CONFIG_NAME = 'resnet_flexible_branched_{'_'.join(INPUT_TYPES)}'")
print("model = hyperparameter_configs[MODEL_CONFIG_NAME]()")
print()

# Example 2: Concatenated architecture
print("# Example 2: Concatenated architecture")
print(f"MODEL_CONFIG_NAME = 'resnet_flexible_concatenated_{'_'.join(INPUT_TYPES)}'")
print("model = hyperparameter_configs[MODEL_CONFIG_NAME]()")
print()

# Example 3: No transfer learning
print("# Example 3: No transfer learning")
print(f"MODEL_CONFIG_NAME = 'resnet_flexible_no_transfer_{'_'.join(INPUT_TYPES)}'")
print("model = hyperparameter_configs[MODEL_CONFIG_NAME]()")
print()

print("Benefits of using hyperparameter configurations:")
print("1. Easy switching between architectures")
print("2. Consistent model creation")
print("3. Automatic naming based on input types")
print("4. Easy to add new configurations")
print()

# You can also create custom configurations
print("Custom configuration example:")
print("def custom_model_config(input_types):")
print("    # Your custom model building logic here")
print("    model = build_resnet_model_flexible(input_types, with_transfer_learning=True)")
print("    # Custom compilation")
print("    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), ...)")
print("    return model")
print()
print("# Add to configurations:")
print("hyperparameter_configs['custom_model'] = lambda: custom_model_config(INPUT_TYPES)")


In [ ]:
# Example: Hyperparameter tuning across different model configurations
# This cell shows how to test multiple model architectures

def train_and_evaluate_model(config_name, hyperparameter_configs, train_dataset, val_dataset, val_patients, INPUT_TYPES):
    """Train and evaluate a single model configuration"""
    print(f"\n=== Training {config_name} ===")
    
    # Build model
    model = hyperparameter_configs[config_name]()
    
    # Early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', mode='max', patience=10, restore_best_weights=True
    )
    
    # Train model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,  # Reduced for faster testing
        callbacks=[early_stopping],
        verbose=0  # Silent training
    )
    
    # Evaluate on validation set
    patient_to_true_labels, patient_to_predicted_probs = plotAndReturnValidationTesting_flexible(val_patients, model, INPUT_TYPES)
    
    # Calculate AUC
    true_labels, predicted_probs = [], []
    for patient_id in patient_to_predicted_probs:
        for prob in patient_to_predicted_probs[patient_id]:
            true_labels.append(patient_to_true_labels[patient_id])
            predicted_probs.append(prob)
    
    val_auc = roc_auc_score(true_labels, predicted_probs)
    print(f"Validation AUC: {val_auc:.4f}")
    
    return val_auc, model

# Example usage (commented out to avoid long execution)
print("=== Hyperparameter Tuning Example ===")
print()
print("To test multiple model configurations:")
print()
print("# Get all configurations")
print("hyperparameter_configs = create_hyperparameter_configs(INPUT_TYPES)")
print()
print("# Test each configuration")
print("results = {}")
print("for config_name in hyperparameter_configs.keys():")
print("    auc, model = train_and_evaluate_model(config_name, hyperparameter_configs, train_dataset, val_dataset, val_patients, INPUT_TYPES)")
print("    results[config_name] = auc")
print()
print("# Find best configuration")
print("best_config = max(results, key=results.get)")
print(f"print(f'Best configuration: {{best_config}} with AUC: {{results[best_config]:.4f}}')")
print()
print("This approach allows you to:")
print("1. Test multiple architectures automatically")
print("2. Compare performance across configurations")
print("3. Select the best performing model")
print("4. Easily add new configurations to test")


In [ ]:
# Example usage of the flexible input system with hyperparameter configurations
# To change input types, simply modify the INPUT_TYPES list at the top of the notebook

print("=== Flexible Input System Usage Examples ===")
print()
print("To use different combinations of inputs, change the INPUT_TYPES variable:")
print()
print("# Only B-mode images:")
print("INPUT_TYPES = ['B_mode']")
print()
print("# B-mode + MBF images:")
print("INPUT_TYPES = ['B_mode', 'MBF']")
print()
print("# B-mode + SI images:")
print("INPUT_TYPES = ['B_mode', 'SI']")
print()
print("# MBF + SI images:")
print("INPUT_TYPES = ['MBF', 'SI']")
print()
print("# All three input types:")
print("INPUT_TYPES = ['B_mode', 'MBF', 'SI']")
print()
print("The system will automatically:")
print("- Load only the specified input types")
print("- Create appropriate model architectures")
print("- Handle data augmentation for multiple inputs")
print("- Generate appropriate evaluation plots")
print()
print("Current configuration:")
print(f"Selected input types: {INPUT_TYPES}")
print(f"Number of input types: {len(INPUT_TYPES)}")
print()
print("Available model architectures (via hyperparameter configurations):")
print("1. Branched architecture: Each input processed separately, then features concatenated")
print("2. Concatenated architecture: Inputs concatenated first, then processed together")
print("3. No transfer learning: Train from scratch without pre-trained weights")
print()
print("How to use hyperparameter configurations:")
print("1. Get configurations: hyperparameter_configs = create_hyperparameter_configs(INPUT_TYPES)")
print("2. Choose model: MODEL_CONFIG_NAME = 'resnet_flexible_branched_B_mode_MBF_SI'")
print("3. Build model: model = hyperparameter_configs[MODEL_CONFIG_NAME]()")
print("4. Train and evaluate as usual")
print()
print("Benefits of hyperparameter configurations:")
print("- Easy switching between architectures")
print("- Consistent model creation")
print("- Automatic naming based on input types")
print("- Easy to add new configurations")
print("- Perfect for hyperparameter tuning")
